In [49]:
# Convert data to python arrays

import numpy as np
import os
from enum import Enum

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

NO_MOVE = -1000

boards = []
y = []

def read_line(string):
    values = string.split(",")
    moves_string = values[0]
    evaluations = [int(values[i]) for i in range(1, 8)]
    indexes = [0 for _ in range(7)]
    matrix = np.full((7, 6), BoardFields.EMPTY.value, dtype="int8")
    turn = BoardFields.PLAYER1

    for move_string in moves_string:
        move = int(move_string) - 1
        matrix[move, indexes[move]] = turn.value
        indexes[move] += 1
        turn = (
            BoardFields.PLAYER1 if turn == BoardFields.PLAYER2 else BoardFields.PLAYER2
        )

    for i in range(len(evaluations)):
        if evaluations[i] == NO_MOVE:
            continue
        child_matrix = np.copy(matrix)
        child_matrix[i, indexes[i]] = turn.value
        boards.append(child_matrix)
        y.append(evaluations[i])


for filename in os.listdir(os.getcwd() + "/data"):
    path = os.getcwd() + "/data/" + filename
    try:
        with open(path, "r") as file:
            for line in file:
                read_line(line.strip())
    except FileNotFoundError:
        print("File not found.")
    except IOError:
        print("Error reading the file.")

np.save(os.getcwd() + "/boards.npy", boards)
np.save(os.getcwd() + "/y.npy", y)


In [ ]:
# Convert python arrays into model input

import numpy as np
import os
from enum import Enum

from convert_board import convert_board

class BoardFields(Enum):
    EMPTY = 0
    PLAYER1 = 1
    PLAYER2 = 2

boards = np.load(os.getcwd() + "/boards.npy")
x = []

length = len(boards)
increment = 5
count = 0
percentage = 0
for board in boards:
    converted_board = convert_board(board)
    x.append(converted_board)
    count += 1
    if count > percentage / 100 * length:
        percentage += increment
        print(percentage, "%")


np.save(os.getcwd() + "/x.npy", x)


In [ ]:
# Create model

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from datetime import datetime

model = tf.keras.Sequential(
    [
        Dense(units=1, input_shape=[2, 3]),
        Dense(units=12, input_shape=[1]),
        Dense(units=6, input_shape=[1]),
        Dense(units=2, input_shape=[1]),
        Dense(units=1, input_shape=[1]),
    ]
)
model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
model.summary()

model.save(f"models/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")


In [ ]:
# Train model

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
from os import listdir
from os.path import isfile, join

x = np.load("x.npy")
y = np.load("y.npy")

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=42
)

while True:
    model_name = sorted([f for f in listdir("models") if isfile(join("models", f))])[-1]
    model = tf.keras.models.load_model(f"models/{model_name}")
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mae"])
    model.fit(
        x_train, y_train, epochs=1, batch_size=64, validation_data=(x_test, y_test)
    )
    model.save(f"models/{datetime.today().strftime('%Y-%m-%d_%H-%M-%S')}.keras")
